In [1]:
import pandas as pd
from PopSynthesis.Methods.GNN_activity.utils import visualize_pyg_graph_with_zones, save_graphs, construct_starting_graph_pyg, add_travel_diaries_to_graph

In [2]:
# Reload the datasets
zones_df = pd.read_csv("data/zones.csv")
purposes_df = pd.read_csv("data/purposes.csv")
sample_households_df = pd.read_csv("data/sample_households.csv")
sample_people_df = pd.read_csv("data/sample_people.csv")
od_matrix_df = pd.read_csv("data/od_matrix.csv")
sample_travel_diaries_df = pd.read_csv("data/sample_travel_diaries.csv")

In [3]:
# Construct the PyG-compatible heterogeneous graph (CPU mode only)
starting_graph_pyg = construct_starting_graph_pyg(zones_df, purposes_df, sample_households_df, sample_people_df, od_matrix_df)

# Display the graph structure
starting_graph_pyg

HeteroData(
  zone={ x=[5, 1] },
  purpose={ x=[30, 1] },
  household={ x=[7, 2] },
  person={ x=[21, 1] },
  (zone, travel, zone)={ edge_index=[2, 10] },
  (zone, has_purpose, purpose)={ edge_index=[2, 30] },
  (household, located_in, zone)={ edge_index=[2, 7] },
  (person, belongs_to, household)={ edge_index=[2, 21] },
  (person, related_to, person)={ edge_index=[2, 46] },
  (person, performs, purpose)={
    edge_index=[2, 0],
    duration=[0],
    ranking=[0],
    joint_activity=[0],
  }
)

In [4]:
# # Map indices to real-world names
# person_id_map = {i: p_id for i, p_id in enumerate(sample_people_df["person_id"])}
# household_id_map = {i: h_id for i, h_id in enumerate(sample_households_df["household_id"])}
# purpose_id_map = {i: f"{row['purpose_type']}_Z{row['zone_id']}" for i, row in purposes_df.iterrows()}

# def get_node_name(node_type, index):
#     if node_type == "person":
#         return person_id_map.get(index, f"Person_{index}")
#     elif node_type == "household":
#         return household_id_map.get(index, f"Household_{index}")
#     elif node_type == "purpose":
#         return purpose_id_map.get(index, f"Purpose_{index}")
#     return f"{node_type}_{index}"

# # Print edges with actual names
# for edge_type in starting_graph_pyg.edge_types:
#     src_type, _, dst_type = edge_type
#     edge_index = starting_graph_pyg[edge_type].edge_index.numpy()

#     print(f"🔹 {edge_type} Edges:")
#     for src, dst in zip(edge_index[0], edge_index[1]):
#         print(f"  {get_node_name(src_type, src)} → {get_node_name(dst_type, dst)}")


In [5]:
# Generate an interactive PyVis visualization with Zones, Households, People, and Purposes correctly labeled
net_visual_zones = visualize_pyg_graph_with_zones(starting_graph_pyg, sample_people_df, sample_households_df, purposes_df, zones_df)
net_visual_zones.save_graph("data/graph_starting.html")

In [6]:
# Generate ID mappings
person_id_map = {p_id: i for i, p_id in enumerate(sample_people_df["person_id"])}
purpose_id_map = {p_id: i for i, p_id in enumerate(purposes_df["purpose_id"])}

# Construct the expected output graph
expected_graph_pyg = add_travel_diaries_to_graph(starting_graph_pyg, sample_travel_diaries_df, person_id_map, purpose_id_map)
expected_graph_pyg

HeteroData(
  zone={ x=[5, 1] },
  purpose={ x=[30, 1] },
  household={ x=[7, 2] },
  person={ x=[21, 1] },
  (zone, travel, zone)={ edge_index=[2, 10] },
  (zone, has_purpose, purpose)={ edge_index=[2, 30] },
  (household, located_in, zone)={ edge_index=[2, 7] },
  (person, belongs_to, household)={ edge_index=[2, 21] },
  (person, related_to, person)={ edge_index=[2, 46] },
  (person, performs, purpose)={
    edge_index=[2, 0],
    duration=[0],
    ranking=[0],
    joint_activity=[0],
  }
)

In [7]:
# Generate an interactive PyVis visualization with Zones, Households, People, and Purposes correctly labeled
net_results = visualize_pyg_graph_with_zones(expected_graph_pyg, sample_people_df, sample_households_df, purposes_df, zones_df)
net_results.save_graph("data/graph_results.html")

In [8]:
save_graphs(starting_graph_pyg, output_dir="data/", output_name="start_graph", output_type="torch")
save_graphs(expected_graph_pyg, output_dir="data/", output_name="expected_graph", output_type="torch")

In [9]:
to_predict_hh_df = pd.read_csv("data/to_predict_households.csv")
to_predict_people_df = pd.read_csv("data/to_predict_people.csv")

In [10]:
to_predict_graphs = construct_starting_graph_pyg(zones_df, purposes_df, to_predict_hh_df, to_predict_people_df, od_matrix_df)
to_predict_graphs

HeteroData(
  zone={ x=[5, 1] },
  purpose={ x=[30, 1] },
  household={ x=[16, 2] },
  person={ x=[52, 1] },
  (zone, travel, zone)={ edge_index=[2, 10] },
  (zone, has_purpose, purpose)={ edge_index=[2, 30] },
  (household, located_in, zone)={ edge_index=[2, 16] },
  (person, belongs_to, household)={ edge_index=[2, 52] },
  (person, related_to, person)={ edge_index=[2, 132] },
  (person, performs, purpose)={
    edge_index=[2, 0],
    duration=[0],
    ranking=[0],
    joint_activity=[0],
  }
)

In [11]:
save_graphs(to_predict_graphs, output_dir="data/", output_name="to_predict_graph", output_type="torch")